<a href="https://colab.research.google.com/github/roudhasalem8/UAE-Construction-Helmet-Violation-Detection-YOLOv8/blob/main/Helmet_Detection_f_(updated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Smart Helmet Detection System for UAE Construction Sites
#**Real-Time Safety Compliance Using YOLOv8 | UAE Vision 2030 & SDG 8**

In [ ]:
# Install required packages
!pip install ultralytics roboflow supervision albumentations opencv-python-headless matplotlib -q


print("All dependencies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 68.9 MB/s eta 0:00:00
All dependencies installed!


In [ ]:
import os
import cv2
import numpy as numpy
import matplotlib.pyplot as plt
from ultralytics import YOLO
from roboflow import Roboflow
import supervision as sv
import albumentations as A
from IPython.display import Image, display
import torch

print("Libraries imported successfully!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Libraries imported successfully!


In [ ]:
# === FINAL FIX: Clean & Extract YOLOv8 Dataset from Google Drive ===

import zipfile
import os
import shutil

# === STEP 0: Mount Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === STEP 1: Set your dataset path from Drive ===
# 👇 Replace the path inside quotes with your actual file path in Drive
drive_zip_path = "/content/drive/MyDrive/Hard Hat Workers.v3-v3.yolov8.zip"

# Copy it to /content for faster read/write
local_zip_path = "/content/Hard Hat Workers.v3-v3.yolov8.zip"
shutil.copy(drive_zip_path, local_zip_path)
print(f"Copied dataset zip from Drive to Colab: {local_zip_path}")

# === STEP 2: Remove any old/bad 'dataset' folder ===
if os.path.exists("dataset"):
    if os.path.isfile("dataset"):
        print("Removing old 'dataset' file...")
        os.remove("dataset")
    else:
        print("Removing old 'dataset' folder...")
        shutil.rmtree("dataset")

# === STEP 3: Extract to temp folder ===
extract_dir = "temp_extracted"
if os.path.exists(extract_dir):
    shutil.rmtree(extract_dir)
os.makedirs(extract_dir, exist_ok=True)

print("Extracting zip... please wait (~1 min for 785MB)")
with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Extracted to:", extract_dir)

# === STEP 4: Find the actual dataset folder inside ===
extracted_items = os.listdir(extract_dir)
print("Items in extract:", extracted_items)

dataset_root = None
for item in extracted_items:
    item_path = os.path.join(extract_dir, item)
    if os.path.isdir(item_path):
        subdirs = os.listdir(item_path)
        if any(x in subdirs for x in ['train', 'valid', 'test']):
            dataset_root = item_path
            break

if dataset_root is None:
    print("⚠️ Could not find dataset root. Moving all contents...")
    dataset_root = extract_dir  # fallback

# === STEP 5: Move to /content/dataset ===
shutil.move(dataset_root, "dataset")
if dataset_root != extract_dir:
    shutil.rmtree(extract_dir)

dataset_location = "dataset"

print("\n🎯 SUCCESS: Dataset ready at /content/dataset")
print("\n📁 Structure:")
!ls -la dataset/

print("\n🖼️ Sample Images:")
!ls dataset/train/images 2>/dev/null | head -5 || echo "No images"

print("\n🏷️ Sample Labels:")
!head -n 2 dataset/train/labels/*.txt 2>/dev/null | head -5 || echo "No labels"


Mounted at /content/drive
Copied dataset zip from Drive to Colab: /content/Hard Hat Workers.v3-v3.yolov8.zip
Extracting zip... please wait (~1 min for 785MB)
✅ Extracted to: temp_extracted
Items in extract: ['README.dataset.txt', 'README.roboflow.txt', 'data.yaml', 'test', 'train']
⚠️ Could not find dataset root. Moving all contents...

🎯 SUCCESS: Dataset ready at /content/dataset

📁 Structure:
total 28
drwxr-xr-x 4 root root 4096 Nov 15 21:20 .
drwxr-xr-x 1 root root 4096 Nov 15 21:20 ..
-rw-r--r-- 1 root root  279 Nov 15 21:20 data.yaml
-rw-r--r-- 1 root root 4025 Nov 15 21:20 README.dataset.txt
-rw-r--r-- 1 root root 1162 Nov 15 21:20 README.roboflow.txt
drwxr-xr-x 4 root root 4096 Nov 15 21:20 test
drwxr-xr-x 4 root root 4096 Nov 15 21:20 train

🖼️ Sample Images:
000001_jpg.rf.063823aaf332982f3e950651f452a290.jpg
000001_jpg.rf.0f6de3cb4d6f36215d869bbf4d230577.jpg
000001_jpg.rf.88f7e69ab4248790a4806cef2d956ecc.jpg
000002_jpg.rf.4b6174111f467b069d247047a2b5e689.jpg
000002_jpg.rf.892f

In [ ]:
# === STEP 1: CREATE valid FOLDER ===
import os, shutil

src_img = "dataset/test/images"
src_lbl = "dataset/test/labels"
dst_img = "dataset/valid/images"
dst_lbl = "dataset/valid/labels"

os.makedirs(dst_img, exist_ok=True)
os.makedirs(dst_lbl, exist_ok=True)

# copy everything from test → valid
!cp -r {src_img}/* {dst_img}/ 2>/dev/null
!cp -r {src_lbl}/* {dst_lbl}/ 2>/dev/null

print(f"valid folder created")
print(f"   images : {len(os.listdir(dst_img))}")
print(f"   labels : {len(os.listdir(dst_lbl))}")

valid folder created
   images : 1766
   labels : 1766


In [ ]:
# === STEP 2: WRITE data.yaml (FULL PATH) ===
data_yaml = """
path: /content/dataset
train: train/images
val: valid/images
test: test/images

names:
  0: person
  1: head
  2: helmet
"""

with open("/content/dataset/data.yaml", "w") as f:
    f.write(data_yaml.strip() + "\n")

print("data.yaml written to /content/dataset/data.yaml")

data.yaml written to /content/dataset/data.yaml


In [ ]:
# === STEP 3: REMOVE WRONG data.yaml (if any) ===
!rm -f /content/data.yaml

In [ ]:
# Runtime → Change runtime type → Hardware accelerator → **GPU**
# Then run this cell to confirm
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")

GPU available: False
GPU name: N/A


In [ ]:
import os

# --- Configuration (Check your Roboflow download path) ---
LABEL_DIR = '/content/dataset/train/labels'
# Assuming your existing labels are all using one ID, e.g., '1',
# but they represent the head WITH a helmet.
OLD_ID = '1'
NEW_ID = '0' # Assuming 'head' is 0 and 'helmet' is 1, let's fix the data based on logic.

# Based on your data.yaml: names: ['head', 'helmet']
# We must assume the current labels are for the object of interest: 'helmet' (ID 1).
# If your current labels are all ID '1', and that represents 'helmet',
# then you only need to create the 'head' (ID 0) labels.

# Since you can't modify all labels, we'll try a different tactic:
# ASSUME ALL EXISTING BOUNDING BOXES ARE FOR THE 'helmet' CLASS (ID 1).

# --- Fix 1: Ensure all labels are correctly set to 'helmet' (ID 1) ---
print(f"Starting to ensure all existing labels are set to 'helmet' (ID 1)...")
count = 0

for filename in os.listdir(LABEL_DIR):
    if filename.endswith(".txt"):
        filepath = os.path.join(LABEL_DIR, filename)

        with open(filepath, 'r') as f:
            lines = f.readlines()

        new_lines = []
        modified = False

        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] != '1': # If the ID is NOT 1 (helmet)
                # Change it to 1 (helmet)
                parts[0] = '1'
                new_lines.append(' '.join(parts) + '\n')
                modified = True
            else:
                new_lines.append(line)

        if modified:
            count += 1
            with open(filepath, 'w') as f:
                f.writelines(new_lines)

print(f"Finished. Modified {count} label files to Class ID 1 ('helmet').")
print("-" * 50)
print("The primary challenge now is the 'head' (ID 0) class...")

Starting to ensure all existing labels are set to 'helmet' (ID 1)...
Finished. Modified 0 label files to Class ID 1 ('helmet').
--------------------------------------------------
The primary challenge now is the 'head' (ID 0) class...


In [ ]:

transform = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.6),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.GaussNoise(var_limit=(10.0, 60.0), p=0.4),
    A.MotionBlur(blur_limit=5, p=0.3),
    A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, alpha_coef=0.2, p=0.3),
    A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0.0, angle_upper=1.0, p=0.2),
    A.RandomShadow(shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=0.3),
    A.RandomRain(drop_length=15, drop_width=1, blur_value=3, p=0.1),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
],
    bbox_params=A.BboxParams(format='yolo',
                             label_fields=['class_labels'],
                             min_visibility=0.3)
)

print("UAE Augmentation Pipeline READY – ZERO WARNINGS")

UAE Augmentation Pipeline READY – ZERO WARNINGS


In [ ]:
# === STEP 5: TRAIN YOLOv8s ===
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(
    data="/content/dataset/data.yaml",   # <-- FULL PATH
    epochs=30,
    imgsz=640,
    batch=32,
    name="uae_helmet_yolov8s",
    project="runs/uae_helmet",
    exist_ok=True,
    patience=15,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.1,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    box=7.5, cls=0.5, dfl=1.5,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=15, translate=0.1, scale=0.5,
    shear=2.0, perspective=0.0,
    flipud=0.0, fliplr=0.5,
    mosaic=1.0, mixup=0.1, copy_paste=0.1,
    auto_augment='randaugment',
    erasing=0.4,
    save=True, plots=True, val=True,
    augment=True               # built‑in YOLO augmentations
)

print("Training finished!")

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=15, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=uae_helmet_yolov8s, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=15, perspective=0.0, plots=True, pose=12.

In [ ]:
# Validate the trained model
metrics = model.val()

print(f"\n{'='*50}")
print(f"FINAL EVALUATION RESULTS")
print(f"{'='*50}")
print(f"mAP@50-95: {metrics.box.map:.3f}")
print(f"mAP@50:    {metrics.box.map50:.3f}")
print(f"mAP@75:    {metrics.box.map75:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall:    {metrics.box.mr:.3f}")
print(f"F1-Score:  {2 * metrics.box.mp * metrics.box.mr / (metrics.box.mp + metrics.box.mr + 1e-16):.3f}")

In [ ]:
# -------------------------------------------------
# 1. Load the best checkpoint
# -------------------------------------------------
from ultralytics import YOLO
import supervision as sv   # for drawing boxes / labels

best_model = YOLO("runs/uae_helmet/uae_helmet_yolov8s/weights/best.pt")
print("Best model loaded")

In [ ]:
# -------------------------------------------------
# 2. Validation – prints mAP, Precision, Recall, F1
# -------------------------------------------------
metrics = best_model.val(data="/content/dataset/data.yaml")   # same yaml as training

print("\n" + "="*55)
print("FINAL EVALUATION RESULTS")
print("="*55)
print(f"mAP@50-95 : {metrics.box.map:.3f}")
print(f"mAP@50    : {metrics.box.map50:.3f}")
print(f"mAP@75    : {metrics.box.map75:.3f}")
print(f"Precision : {metrics.box.mp:.3f}")
print(f"Recall    : {metrics.box.mr:.3f}")
print(f"F1‑Score  : {2*metrics.box.mp*metrics.box.mr/(metrics.box.mp+metrics.box.mr+1e-16):.3f}")

In [ ]:
# -------------------------------------------------
# 3. Download a realistic UAE construction picture
# -------------------------------------------------
#!wget -q "https://upload.wikimedia.org/wikipedia/commons/2/2d/Construction_workers_road.jpg" -O test_uae.jpg
#print("Test image saved as test_uae.jpg")

In [ ]:
# -------------------------------------------------
# 4. Run inference & draw boxes + class‑confidence
# -------------------------------------------------
import cv2
import matplotlib.pyplot as plt

# inference
results = best_model("test2.jpg")[0]

# supervision helpers
detections = sv.Detections.from_ultralytics(results)

# labels: class name + confidence
labels = [
    f"{best_model.names[int(cls)]} {conf:.2f}"
    for cls, conf in zip(detections.class_id, detections.confidence)
]

# draw
annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_thickness=1, text_scale=0.5)

annotated = annotator.annotate(scene=results.orig_img.copy(), detections=detections)
annotated = label_annotator.annotate(scene=annotated, detections=detections, labels=labels)

sv.plot_image(annotated, (12, 8))

In [ ]:
# -------------------------------------------------
# 5. Detect workers whose head is NOT covered by a helmet
# -------------------------------------------------
detections = sv.Detections.from_ultralytics(results)
img = results.orig_img.copy()

helmet_boxes = []
head_boxes   = []

# separate helmet & head detections
for cls, box in zip(detections.class_id, detections.xyxy):
    if best_model.names[int(cls)] == "helmet":
        helmet_boxes.append(box)
    elif best_model.names[int(cls)] == "head":
        head_boxes.append(box)

no_helmet_workers = []

for head_box in head_boxes:
    hx1, hy1, hx2, hy2 = head_box
    head_cx = (hx1 + hx2) / 2
    head_cy = (hy1 + hy2) / 2

    has_helmet = False
    for helm_box in helmet_boxes:
        hx1h, hy1h, hx2h, hy2h = helm_box
        if (hx1h < head_cx < hx2h) and (hy1h - 60 < head_cy < hy2h + 60):
            has_helmet = True
            break

    if not has_helmet:
        no_helmet_workers.append(head_box)

# draw alerts
alert_img = img.copy()
for box in no_helmet_workers:
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(alert_img, (x1, y1), (x2, y2), (0, 0, 255), 3)
    cv2.putText(alert_img, "NO HELMET!", (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

print(f"ALERT: {len(no_helmet_workers)} worker(s) WITHOUT helmet")
sv.plot_image(alert_img, (12, 8))

In [ ]:
# -------------------------------------------------
# 6. Export ONNX (edge) + TorchScript (fallback)
# -------------------------------------------------
best_model.export(format="onnx", imgsz=640, simplify=True)
best_model.export(format="torchscript")

print("Model exported")
!ls -lh runs/uae_helmet/uae_helmet_yolov8s/weights/

In [ ]:
# -------------------------------------------------
# 7. Zip & download everything you need for the report
# -------------------------------------------------
!zip -r uae_helmet_results.zip \
    runs/uae_helmet/uae_helmet_yolov8s/weights/best.pt \
    runs/uae_helmet/uae_helmet_yolov8s/weights/best.onnx \
    runs/uae_helmet/uae_helmet_yolov8s/weights/best.torchscript \
    test_uae.jpg

from google.colab import files
files.download("uae_helmet_results.zip")